In [7]:
import numpy as np
import math
import torch # Used only for matrix multiplication
from torch.autograd import Variable
import pandas as pd
import gc
import tensorflow as tf # Used only to acquire MNIST data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
X = []
device = torch.device("cpu")
# Probably an input of images (eg MNIST or something)
num_layers = 4
dtype = torch.float
torch.seed = 9
np.random.seed(9)

# Convolutional Neural Network written from scratch for basic classification tasks (eg MNIST / Cats and Dogs)
# Can be used as a discriminator for a GAN as well

# CNN_Engine is sort of complete. Now you have to clean up the code and make it bug free 
# Also, further optimisation to improve speed would be good. Use dictionaries wherever possible

class CNN_Engine:
    def __init__(self,num_layers,X,y,loss_type,num_epochs,bs,stride,dropout):
        self.num_layers = num_layers
        self.X = X
        self.y = y
        self.loss_type = loss_type      
        self.num_epochs = num_epochs
        self.bs = bs
        self.padding = padding
        self.stride = stride
        self.dropout = dropout
        
    def cost_function(self,y_pred,y): # Assuming cross entropy loss
        sumx = 0
        for k in self.y:
            sumx += -k*math.log(y_pred)
        return sumx

    def Relu(self,x):
        if self.x > 0:
            return self.x
        else:
            return 0
    
    def MaxPool2d(self,X,kernel_size,stride):
        return

    def BatchNorm(self,X):
        return (X-torch.mean(X)) / torch.std(X)
    
    def im2col2d(X,kernel,stride): # Assuming X and the kernel are 2D inputs
        # We transform the input matrix X into a matrix Ω(x) such that the columns correspond to the 
        # set of elements that will be multiplied by the kernel in each sliding convolution operation
        # Given x -> Ω(x), conv2d(x,kernel) = Ω(x)' * kernel where ' denotes a transpose operation

        X = X.detach().numpy()
        X = np.array(X)
        kernel_shape = np.array(list(np.shape(kernel))).T
        img_shape = np.array(list(np.shape(X))).T
        a = np.empty((kernel_shape[0]*kernel_shape[1]*kernel_shape[2]))
        output_matrix = np.reshape(a,(kernel_shape[0]*kernel_shape[1]*kernel_shape[2],1))
        for k in range(0,img_shape[2],stride):
             for j in range(0,img_shape[1],stride):
                for i in range(0,img_shape[0],stride):
                        try:
                            vals = X[i:i+kernel_shape[0],j:j+kernel_shape[1],k:k+kernel_shape[2]]
                            vals = np.reshape(vals,(int(np.shape(vals)[2]*np.shape(vals)[1]*np.shape(vals)[0]),1))
                            output_matrix = np.hstack((output_matrix,vals))
                        except:
                            continue
        
        output_matrix = np.delete(output_matrix,0,1)
        output_matrix = torch.FloatTensor(output_matrix)
        return torch.nn.Parameter(output_matrix)

    def conv2D(self,channels,X,stride,kernel_shape,padding = False): # filters = dimensionality of output space
            # If padding is enabled, we pad the input with zeros such that the input size 
            # remains the same if weights with stride 1 are applied to the input
            kernel = np.random.uniform(-0.1,0.1,size = (kernel_shape[0],kernel_shape[1],kernel_shape[2])) # Our input
            kernel = torch.FloatTensor(kernel)
            kernel.requires_grad = True
            X = X.detach().numpy()
            if padding:
                if kernel_shape[1] % 2 == 0 and kernel_shape[2] % 2 == 0:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2)-1,math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2)-1)), 'symmetric')
                elif kernel_shape[1] % 2 != 0 and kernel_shape[2] % 2 == 0:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2),math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2)-1)), 'symmetric')
                elif kernel_shape[1] % 2 == 0 and kernel_shape[2] % 2 != 0:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2)-1,math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2))), 'symmetric')
                else:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2),math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2))), 'symmetric')
            X = torch.FloatTensor(X)
            img_shape = np.transpose(np.array(list(np.shape(X))))
            output_size1 = math.floor((img_shape[1] - kernel_shape[1])/(stride)) + 1
            output_size2 = math.floor((img_shape[2] - kernel_shape[2])/(stride)) + 1
            output_shape = [channels,output_size1,output_size2]
            X_im2col = im2col2d(X,kernel,stride)
            
            weight = torch.reshape(kernel,(kernel_shape[0]*kernel_shape[1]*kernel_shape[2],1))
            # weight consists of only one weight vector. But the dimensionality of output space has to be 
            # num_filters. So we need to stack weight vectors horizontally and create num_filters number of 
            # feature maps
            for i in range(channels-1):
                weight2 = np.random.uniform(-0.1,0.1,size = (kernel_shape[0]*kernel_shape[1]*kernel_shape[2],1)) # Our input
                weight2 = torch.FloatTensor(weight2)
                weight2.requires_grad = True 
                weight = torch.cat((weight2, weight),1) # do this num_filters - 1 number of times
            conv_output = torch.t(X_im2col).mm(weight)
            conv_output = torch.reshape(conv_output,(output_shape))
            weight.retain_grad()
            return torch.nn.Parameter(conv_output), torch.nn.Parameter(weight),X_im2col

    def backprop_with_gradient_descent(self,y_pred,y,x):
        return

    def feedforward(self,x):
        
        return
    
    
# I havent converted the code below into a CNN_Engine class, because the code below is for experimentation 
# The CNN trains, but it would be better to make it library-esque so that it can be run like PyTorch
    
def im2col2d(X,kernel,stride,im_needed = True,shape_specified = False): # Assuming X and the kernel are 2D inputs
        # We transform the input matrix X into a matrix Ω(x) such that the columns correspond to the 
        # set of elements that will be multiplied by the kernel in each sliding convolution operation
        # Given x -> Ω(x), conv2d(x,kernel) = Ω(x)' * kernel where ' denotes a transpose operation

        X = X.detach().numpy()
        X = np.array(X)
        if not shape_specified: # If the kernel shape is not specified, then it is assumed that the argument kernel is a weight matrix
            kernel_shape = kernel.shape
        else: # If the kernel shape is specified, then it is assumed that the argument is the shape of the weight, not the weight itself
            kernel_shape = kernel

        img_shape = np.shape(X)
        a = np.empty((kernel_shape[0]*kernel_shape[1]*kernel_shape[2]))
        output_matrix = np.reshape(a,(kernel_shape[0]*kernel_shape[1]*kernel_shape[2],1))
        q = 0
        output_size1 = math.floor((img_shape[1] - kernel_shape[1])/(stride)) + 1
        output_size2 = math.floor((img_shape[2] - kernel_shape[2])/(stride)) + 1
        if im_needed:
            indicator_dict = {}            # Map (i,j,k) to (p,q)  (i,j,k): (p,q)
        coord_matrix = np.zeros((img_shape[0],img_shape[1],img_shape[2]),dtype='object')
        for k in range(0,img_shape[2]):
             for j in range(0,img_shape[1]):
                for i in range(0,img_shape[0]):
                    coord_matrix[i,j,k] = (i,j,k)
                    
        for k in range(0,img_shape[2],stride):
             for j in range(0,img_shape[1],stride):
                for i in range(0,img_shape[0],stride):
                        try:
                            vals = X[i:i+kernel_shape[0],j:j+kernel_shape[1],k:k+kernel_shape[2]]
                            vals = np.reshape(vals,(int(np.shape(vals)[2]*np.shape(vals)[1]*np.shape(vals)[0]),1))
                            output_matrix = np.hstack((output_matrix,vals))
                            indices = coord_matrix[i:i+kernel_shape[0],j:j+kernel_shape[1],k:k+kernel_shape[2]]
                            
                            indices = np.reshape(indices,(int(np.shape(indices)[2]*np.shape(indices)[1]*np.shape(indices)[0])))                            
                            coords = [(p,q) for p in range(len(indices))]
                            d = dict(zip(coords,list(indices)))
                            indicator_dict.update(d)
                            q+=1
                            
                        except:
                             continue
        
        if im_needed:
            reversed_indicator_dict = dict()
            for k,v in indicator_dict.items():
                reversed_indicator_dict.setdefault(v, []).append(k) # Reversed mapping now in order to 
        
        output_matrix = np.delete(output_matrix,0,1)
        output_matrix = torch.FloatTensor(output_matrix)
        if im_needed:
            return torch.nn.Parameter(output_matrix),reversed_indicator_dict
        else:
            return torch.nn.Parameter(output_matrix),-1
    
def conv2D(self,channels,X,stride,kernel_shape,padding = False,initialize_weights = True,*args): # filters = dimensionality of output space
            # If padding is enabled, we pad the input with zeros such that the input size 
            # remains the same if weights with stride 1 are applied to the input
            
            if initialize_weights:
                kernel = np.random.uniform(-0.1,0.1,size = (kernel_shape[0],kernel_shape[1],kernel_shape[2])) # Our input
             # Our input
                kernel = torch.FloatTensor(kernel)
                kernel.requires_grad = True
            else:
                kernel = args[0] # weights and bias must be given if initialise weights is disabled
                bias = args[1]
                kernel_shape = kernel.shape
            
            X = X.detach().numpy()
            if padding: # Can only pad during initialization -> weights and input shapes cannot change during feedforward and backpropagation
                if kernel_shape[1] % 2 == 0 and kernel_shape[2] % 2 == 0:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2)-1,math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2)-1)), 'symmetric')
                elif kernel_shape[1] % 2 != 0 and kernel_shape[2] % 2 == 0:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2),math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2)-1)), 'symmetric')
                elif kernel_shape[1] % 2 == 0 and kernel_shape[2] % 2 != 0:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2)-1,math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2))), 'symmetric')
                else:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2),math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2))), 'symmetric')
            
            X = torch.FloatTensor(X)
            
            img_shape = X.shape
            
            output_size1 = math.floor((img_shape[1] - kernel_shape[1])/(stride)) + 1
            output_size2 = math.floor((img_shape[2] - kernel_shape[2])/(stride)) + 1
            output_shape = [channels,output_size1,output_size2]
            
            X_im2col,im = im2col2d(X,kernel,stride)
            
            
            if initialize_weights:
                weight = torch.reshape(kernel,(kernel_shape[0]*kernel_shape[1]*kernel_shape[2],1))
                # weight consists of only one weight vector. But the dimensionality of output space has to be 
                # num_filters. So we need to stack weight vectors horizontally and create num_filters number of 
                # feature maps
                for i in range(channels-1):
                    weight2 = np.random.uniform(-0.1,0.1,size = (kernel_shape[0]*kernel_shape[1]*kernel_shape[2],1)) # Our input
                    weight2 = torch.FloatTensor(weight2)
                    weight2.requires_grad = True 
                    weight = torch.cat((weight2, weight),1) # do this num_filters - 1 number of times
                conv_output = torch.t(X_im2col).mm(weight)
                bias = torch.Tensor(np.random.uniform(-0.1,0.1,size = conv_output.shape))
                conv_output += bias
                conv_output = torch.reshape(conv_output,(output_shape))
                return torch.nn.Parameter(conv_output), torch.nn.Parameter(weight),X_im2col,im, output_shape,bias
            else:
                # Since weights are already initialised, the relevant channels are already dictated in the architecture. 
                # Therefore, conv output is just a matmul 
                conv_output = torch.t(X_im2col).mm(kernel) + bias
                return torch.nn.Parameter(conv_output),X_im2col
                
                
            

# For example, if you call out.backward() for some variable out that 
# involved x in its calculations then x.grad will hold ∂out/∂x      
# kernel = np.array([[[1,1],[1,1]]]) # format = channels * rows * columns
# X = np.array([[[1,2,3,1],[4,5,6,1],[7,8,9,1]]])
def maxpool(X,kernel_shape,stride):
    
    X_im2col,im = im2col2d(X,kernel_shape,stride,im_needed = True,shape_specified = True)
    output = torch.zeros((X.shape[1],1))
    output_size0 = math.floor((X.shape[0] - kernel_shape[0])/(stride)) + 1
    output_size1 = math.floor((X.shape[1] - kernel_shape[1])/(stride)) + 1
    output_size2 = math.floor((X.shape[2] - kernel_shape[2])/(stride)) + 1

    for j in range(X_im2col.shape[1]):
        output[j] = max(X_im2col[:,j])
    output = torch.reshape(output,(output_size0,output_size1,output_size2))
    return output

class MyReLU(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input
    
def softmax(y): # A bit lengthy for a simple softmax implementation but it was all I could do to avoid gradient related errors
#     y = y.squeeze()
    epsilon = 0.001
    y = y.detach().numpy()
    return torch.Tensor(np.exp(y)/(epsilon + sum(np.exp(y))))

def BatchNorm(X): # (X - mu) / sigma
    epsilon = 0.001  # To prevent overflow and ensure numerical stability
    bn = (X - torch.mean(X)) / (torch.std(X)+epsilon)   
    sigma.append(torch.std(X)+epsilon)
    return bn
    
def conv_bn_relu(X,channels,stride,kernel_shape,padding,activation = 'sigmoid',initialize_weights = True,*args): # Refactoring conv_batchnorm_relu as one layer
    if initialize_weights:
        output_1,weights,X_im2col,im,output_shape,bias = conv2D(0,channels,X,stride,kernel_shape,padding,initialize_weights = True,*args) # conv
    else:
        output1,X_im2col = conv2D(0,channels,X,stride,kernel_shape,padding,initialize_weights = False,*args) # conv
        
    output_1 = BatchNorm(output_1) # batchnorm
    if activation == 'sigmoid':
        X_1 = sigmoid(output_1)
    elif activation == 'relu':
        X_1 = relu(output_1)
    else:
        X_1 = output_1
    if initialize_weights:
        return torch.nn.Parameter(X_1),torch.nn.Parameter(weights),torch.nn.Parameter(X_im2col),im,output_shape,bias
    else:
        return output1,X_im2col

def flatten_and_dense(X,out_channels,activtation = 'sigmoid', initialise_weights = True,*args): 
    """ Flattens the input and outputs a fully connected dense layer after applying an activation function""" 
    shape = X.shape
    X = torch.reshape(X,(shape[0]*shape[1]*shape[2],1)) # Flatten
    if initialise_weights == True:
        weights = torch.Tensor(np.random.uniform(-0.1,0.1, size = (out_channels,len(X))))
        bias = torch.Tensor(np.random.uniform(-0.1,0.1, size = (out_channels,1)))
    else:
        weights = args[0]
        bias = args[1]
    if activation == 'sigmoid':
        output = sigmoid(weights.mm(X) + bias)
    else: # Only support sigmoid and relu for now
        output = relu(weights.mm(X) + bias)
    return output
                                                                     
def cross_entropy(y_pred,y):
        epsilon = 0.001 # To prevent overflow and ensure numerical stability
        return sum(-y*np.log(y_pred+epsilon))

def sigmoid(X):
    X = X.detach().numpy()
    X = torch.FloatTensor((1/(1+(np.exp(-X)))))
    return X

def getIndexes(indicator_matrix, val): 

    ''' Get desired coordinates of val in an indicator matrix. 
    val will be in the form of ijk cartesian coordinates'''    
    try:
        return indicator_matrix[val]
    except:
        return []

relu = MyReLU.apply

padding = False # Mostly will never set this to true
sigma = []
num_examples = 250
# y = np.random.randint(0,2,size = (10,1))
y = np.zeros((10,num_examples))

for i in range(y.shape[1]):
    y[y_train[i],i] = 1
y_input = y   
y = Variable(torch.FloatTensor(y))
y.requires_grad = False

X_input = x_train[0:num_examples,:,:]
X_input = torch.FloatTensor(X_input)

input_dictionary = {}  # Dictionary of input data and their respective im2col keyed by the training sample number

architecture = {}

#### Architecture: Shape it as you want ####
channels = 2

X = X_input[0:1,:,:]
y = torch.Tensor(y_input[:,0:1])

stride = 1
kernel_shape1 = (1,5,5)
X_1,weights1,X_im2col,im1,output_shape1,bias1 = conv_bn_relu(X,channels,stride,kernel_shape1,padding,activation = "relu",initialize_weights = True)

architecture['layer1'] = np.array([X_1,weights1,bias1,X_im2col,im1,output_shape1,kernel_shape1,stride],dtype = 'object')

# # Experimental
# im1 = im1.to_numpy()

# for i in range(im1.shape[0]):
#     for j in range(im1.shape[1]):
#         im1[i,j] = im1[i,j].squeeze().squeeze()
# im1 = im1.squeeze()





stride = 2
kernel_shape2 = (channels,7,7)

X_2,weights2,X_1_im2col,im2,output_shape2,bias2 = conv_bn_relu(X_1,channels,stride,kernel_shape2,padding,initialize_weights = True) # can change the number of channels to any integer value

architecture['layer2'] = np.array([X_2,weights2,bias2,X_1_im2col,im2,output_shape2,kernel_shape2,stride],dtype = 'object')


# # Experimental
# im2 = im2.to_numpy()
# for i in range(im2.shape[0]):
#     for j in range(im2.shape[1]):
#         im2[i,j] = im2[i,j].squeeze().squeeze()
# im2 = im2.squeeze()




stride = 3
kernel_shape3 = (channels,8,8)
a = maxpool(X_2,kernel_shape3,stride)
channels = 10
X_3,weights3,X_2_im2col,im3,output_shape3,bias3 = conv_bn_relu(X_2,channels,stride,kernel_shape3,padding,initialize_weights = True)
architecture['layer3'] = np.array([X_3,weights3,bias3,X_2_im2col,im3,output_shape3,kernel_shape3,stride],dtype = 'object')


print(X_3.shape)
# kernel_shape = (10,1,1)
# X_4,weights4,X_3_im2col,im4 = conv_bn_relu(X_3,channels,stride,kernel_shape,padding)

# kernel_shape = (10,2,2)
# X_5,weights5,X_4_im2col = conv_bn_relu(X_4,channels,stride,kernel_shape,padding)

# kernel_shape = (10,1,1) # no of channels in kernel must be the same as that in the input image, i.e. X_5 in this case
# Y,weights6, X_5_im2col = conv_bn_relu(X_5,channels,stride,kernel_shape,padding)
# Y = X_2
# X_2 = X_2.squeeze()


##### Experimental: Implementing Backpropagation 
y_pred = softmax(X_3)
y_pred = torch.reshape(y_pred,y.size())
loss = Variable(cross_entropy(y_pred.squeeze(),y.squeeze()))
weights2.requires_grad = False
weights1.requires_grad = False
weights3.requires_grad = False
print("Hi")

dropout = 0.2
epochs = 21
bs = 10
arr = []
epsilon = 0.01

beta_1 = 0.9
beta_2 = 0.99

def adam(g,beta_1,beta_2,m,v,t):
    g = g.detach().numpy()
    m = m.detach().numpy()
    v = v.detach().numpy()
    m = beta_1 * m + (1 - beta_1) * g
    v = beta_2 * v + (1 - beta_2) * np.power(g, 2)
    m_hat = m / (1 - np.power(beta_1, t)) + (1 - beta_1) * g / (1 - np.power(beta_1, t))
    v_hat = v / (1 - np.power(beta_2, t))
    grad = lr * m_hat / (np.sqrt(v_hat) + epsilon)
    return torch.Tensor(grad),torch.Tensor(m),torch.Tensor(v)

for epoch in range(epochs): # training + backpropagation. Problem: too slow -> need to speed it up
    clip = 5
    lr_multiplier = 0.2
    stride1 = 1
    stride2 = 2 
    stride3 = 3

    gc.collect()
#     lr = 0.05
    lr = min(0.2,0.01*np.exp(lr_multiplier*epoch))
    
    exp = 200
    y = np.zeros((10,exp))
    x_input = x_test[0:exp,:,:]
    x_input = torch.FloatTensor(x_input)
    val_loss = 0
    num_correct = 0
    for i in range(y.shape[1]):
        y[y_test[i],i] = 1
        y_val = torch.FloatTensor(y)
    for index in range(exp):
        X = BatchNorm(x_input[index,:,:])
        y = np.array(torch.Tensor(y_val[:,index]))
        X = torch.reshape(X,(1,X.shape[0],X.shape[1]))
        # X = torch.reshape(y,(1,X.shape[0],X.shape[1]))
        X_im2col,imX1 = im2col2d(X,kernel_shape1,stride1,im_needed = False,shape_specified = True)
        X_1 = relu(BatchNorm(torch.t(X_im2col).mm(weights1) + bias1))
        X_1 = torch.reshape(X_1,output_shape1)
        X_1_im2col, imX2 = im2col2d(X_1,kernel_shape2,stride2,im_needed = False,shape_specified = True)
        X_2 = relu(BatchNorm(torch.t(X_1_im2col).mm(weights2) + bias2))
        X_2 = torch.reshape(X_2,output_shape2)
        X_2_im2col, imX3 = im2col2d(X_2,kernel_shape3,stride3,im_needed = False,shape_specified = True)
        X_3 = torch.t(X_2_im2col).mm(weights3) + bias3
        X_x = X_3.squeeze()
        y_pred = np.array(softmax(X_x))
        val_loss += cross_entropy(y_pred.squeeze(),y.squeeze())
        prediction = np.argmax(y_pred,axis=0)
        actual = np.argmax(y,axis=0)
    #     imshow(np.array(X.squeeze()))
    #     print("Prediction is: {}".format(prediction))
        if prediction == actual:
            num_correct += 1
    accuracy = num_correct / exp
    accuracy *= 100
    
    arr.append(loss)
    loss = 0
    # forward_propagation
#     for i in architecture:
#         X_i,weightsi,biasi,X_i_im2col,imi,output_shapei,kernel_shapei,stridei = architecture[i]
#         X_i1 = relu(BatchNorm(torch.t(X_i_im2col).mm(weightsi)))
#         X_i1 = torch.reshape(X_i1,output_shapei)
#         architecture[i][0] = X_i1
    t = 1
    
    # Initialising first and second moments for each gradient and bias matrix in each layer
    grad_weights3 = torch.zeros(weights3.size())
    grad_weights2 = torch.zeros(weights2.size())
    grad_weights1 = torch.zeros(weights1.size())
    grad_bias3 = torch.zeros(bias3.size())
    grad_bias2 = torch.zeros(bias2.size())
    grad_bias1 = torch.zeros(bias1.size())
    bias3 = torch.zeros(bias3.size())
    bias2 = torch.zeros(bias2.size())
    bias1 = torch.zeros(bias1.size()) 
    
    m3 = torch.zeros(weights3.size())
    v3 = torch.zeros(weights3.size())
    m2 = torch.zeros(weights2.size())
    v2 = torch.zeros(weights2.size())
    m1 = torch.zeros(weights1.size())
    v1 = torch.zeros(weights1.size())
    m3b = torch.zeros(bias3.size())
    v3b = torch.zeros(bias3.size())
    m2b = torch.zeros(bias2.size())
    v2b = torch.zeros(bias2.size())
    m1b = torch.zeros(bias1.size())
    v1b = torch.zeros(bias1.size())
    
    for j in range(num_examples):
        sigma = []  # Collecting the sigmas from the BatchNorm layer for the purposes of gradient descent
        # Y = (X - mu) /sigma
        # dY_dX = (1/sigma)*ones(X.shape)
        if j%bs == 0 or j==num_examples-1 and j != 0: 
            if j%10 == 0:
                print("Example: {}".format(j))
            # Dropout: Not implemented yet
            
#             grad_weights3 = torch.Tensor(grad_weights3)
#             grad_weights3 = np.array(grad_weights3.detach().numpy())
#             grad_weights2 = np.array(grad_weights2.detach().numpy())
#             grad_weights1 = np.array(grad_weights1.detach().numpy())
#             grad_bias3 = np.array(grad_bias3.detach().numpy())
#             grad_bias2 = np.array(grad_bias2.detach().numpy())
#             grad_bias1 = np.array(grad_bias1.detach().numpy())
#             # Dropout to prevent overfitting
#             indices_weights3 = np.random.choice(np.arange(grad_weights3.size), replace=False, size=int(grad_weights3.size * dropout))
#             indices_weights2 = np.random.choice(np.arange(grad_weights2.size), replace=False, size=int(grad_weights2.size * dropout))
#             indices_weights1 = np.random.choice(np.arange(grad_weights1.size), replace=False, size=int(grad_weights1.size * dropout))
#             indices_bias3 = np.random.choice(np.arange(grad_bias3.size), replace=False, size=int(grad_bias3.size * dropout))
#             indices_bias2 = np.random.choice(np.arange(grad_bias2.size), replace=False, size=int(grad_bias2.size * dropout))
#             indices_bias1 = np.random.choice(np.arange(grad_bias1.size), replace=False, size=int(grad_bias1.size * dropout))
#             print(indices_weights3)
#             grad_weights3[indices_weights3] = 0
#             grad_weights2[indices_weights2] = 0
#             grad_weights1[indices_weights1] = 0
#             grad_bias3[grad_bias3] = 0
#             grad_bias2[grad_bias2] = 0
#             grad_bias1[grad_bias1] = 0
             
            # Adam optimiser
            q3,m3,v3 = adam(grad_weights3/bs,beta_1,beta_2,m3,v3,t)
            q2,m2,v2 = adam(grad_weights2/bs,beta_1,beta_2,m2,v2,t)
            q1,m1,v1 = adam(grad_weights1/bs,beta_1,beta_2,m1,v1,t)
            qb3,m3b,v3b = adam(grad_bias3/bs,beta_1,beta_2,m3b,v3b,t)
            qb2,m2b,v2b = adam(grad_bias2/bs,beta_1,beta_2,m2b,v2b,t)
            qb1,m1b,v1b = adam(grad_bias1/bs,beta_1,beta_2,m1b,v1b,t)

#             qb3 = 0
#             qb2 = 0
#             qb1 = 0
            weights1 -= q1
            bias1 -= qb1
            
            weights2 -= q2
            bias2 -= qb2
            
            bias3 -= qb3
            weights3 -= q3
            
            grad_weights3 = torch.zeros(grad_weights3.size())
            grad_weights2 = torch.zeros(grad_weights2.size())
            grad_weights1 = torch.zeros(grad_weights1.size())
            grad_bias3 = torch.zeros(grad_bias3.size())
            grad_bias2 = torch.zeros(grad_bias2.size())
            grad_bias1 = torch.zeros(grad_bias1.size())
            t+= 1
        
        # Feedforward
        X = BatchNorm(X_input[j:j+1,:,:])
        y = torch.Tensor(y_input[:,j:j+1])
        X_im2col,imX1 = im2col2d(X,kernel_shape1,stride1,im_needed = False,shape_specified = True)
        X_1 = relu(BatchNorm(torch.t(X_im2col).mm(weights1) + bias1))
        X_1 = torch.reshape(X_1,output_shape1)
        X_1_im2col, imX2 = im2col2d(X_1,kernel_shape2,stride2,im_needed = False,shape_specified = True)
        X_2 = relu(BatchNorm(torch.t(X_1_im2col).mm(weights2) + bias2))
        X_2 = torch.reshape(X_2,output_shape2)
        X_2_im2col, imX3 = im2col2d(X_2,kernel_shape3,stride3,im_needed = False,shape_specified = True)
        X_3 = torch.t(X_2_im2col).mm(weights3) + bias3
        X_x = X_3.squeeze()
        y_pred = softmax(X_x)
        
        y_pred = torch.reshape(y_pred,y.size())

        loss += Variable(cross_entropy(y_pred.squeeze(),y.squeeze()),requires_grad=True)

        ## BACKPROPAGATION ##
        
        dz_dX3 = y_pred - y     # Gradient of loss function wrt last layer output before softmax
        
        dz_dX3[dz_dX3 > clip] = 0 # Gradient clipping
        dz_dX3[dz_dX3 < -clip] = 0 # Gradient clipping
#         dz_dX3[torch.t(X_3) <= 0.000001] = 0     # For ReLu

#         dz_dX3 = sigmoid(dz_dX3)*(1-sigmoid(dz_dX3))  # for sigmoid

        dz_db3 = torch.t(dz_dX3)
        # ' = derivative
        # X3 = sigmoid(bn(X2im2col*weights3))
        # dX3_dX2im2col = sigmoid'(weights3)
        dz_dweights3 = (dz_dX3).mm(torch.t(X_2_im2col))    # dz_dweights3 = dz_dX3 * dX3_dweights3  (chain rule)
        dz_dweights3[dz_dweights3 > clip] = 0  # Gradient Clipping
        dz_dweights3[dz_dweights3 < -clip] = 0 # Gradient Clipping
#         dz_dweights3 *= sigma[2]
        gc.collect()

        Y = weights3.mm(dz_dX3) # Y is the same shape as X_2im2col -> dz_dX2 (im2col version) = dz_dX3 * dX3_dX2im2col = Y as dX3_dX2im2col = weights3
        # might need to do sigmoid'(weights3) or weights3()
        
        # dz_dX | (i,j,k) = sum (Y) at all locations (p,q) in X_2im2col where X_2(i,j,k) is placed. Makes sense coz Y is the im2col version of dz_dX2
 
        dz_dX2 = torch.zeros(X_2.size())
        
        for i in range(np.shape(X_2)[0]):
            for j in range(np.shape(X_2)[1]):
                for k in range(np.shape(X_2)[2]):
                    idxs = getIndexes(im3,(i,j,k))
                    dz_dX2[i,j,k] = sum([Y[i[0],i[1]] for i in idxs])
#         dz_dX2[X_2 <= 0.000001] = 0
        dz_dX2 = torch.reshape(dz_dX2,(np.shape(X_2)[1]*np.shape(X_2)[2],-1))
#         dz_dX2[dz_dX2 < 0] = 0        # For ReLu
        
        dz_dX2[dz_dX2 > clip] = 0  # Gradient Clipping
        dz_dX2[dz_dX2 < -clip] = 0 # Gradient Clipping
#         dz_dX2 = sigmoid(dz_dX2)*(1-sigmoid(dz_dX2))    #for sigmoid

        dz_db2 = dz_dX2
        
        # ' = derivative
        # X2 = sigmoid(bn(X1im2col*weights2))
        # dX2_dX1im2col = X1im2col*sigmoid'(weights2)

        dz_dweights2 = X_1_im2col.mm(dz_dX2)    
        dz_dweights2[dz_dweights2 > clip] = 0  # Gradient Clipping
        dz_dweights2[dz_dweights2 < -clip] = 0 # Gradient Clipping
#         dz_dweights2 *= sigma[1]

        Y = weights2.mm(torch.t(dz_dX2))

        dz_dX1 = torch.zeros(X_1.size())

        for i in range(np.shape(X_1)[0]):
            for j in range(np.shape(X_1)[1]):
                for k in range(np.shape(X_1)[2]): 
                    idxs = getIndexes(im2,(i,j,k))

                    dz_dX1[i,j,k] = sum([Y[p[0],p[1]] for p in idxs])

#         dz_dX1[X_1 <= 0.000001] = 0
        dz_dX1 = torch.reshape(dz_dX1,(np.shape(X_1)[1]*np.shape(X_1)[2],-1))

        gc.collect()
        
#         dz_dX1[dz_dX1 < 0] = 0 # For Relu
        
        dz_dX1[dz_dX1 > clip] = 0 # Gradient clipping
        dz_dX1[dz_dX1 < -clip] = 0 # Gradient clipping
        
#         dz_dX1 = sigmoid(dz_dX1)*(1-sigmoid(dz_dX1)) # For sigmoid

        dz_db1 = dz_dX1
        

        dz_dweights1 = X_im2col.mm(dz_dX1)
        dz_dweights1[dz_dweights1 > clip] = 0  # Gradient Clipping
        dz_dweights1[dz_dweights1 < -clip] = 0 # Gradient Clipping
#         dz_dweights1 *= sigma[0]
        
        grad_weights3 += torch.t(dz_dweights3)
        grad_weights2 += dz_dweights2
        grad_weights1 += dz_dweights1
        grad_bias3 += dz_db3
        grad_bias2 += dz_db2
        grad_bias1 += dz_db1
        gc.collect()


    print("Epoch: {}".format(epoch))
    print("Training Loss: {}".format(loss/num_examples)) 
    print("Validation Accuracy: {} %".format(accuracy))
    print("Validation Loss: {} %".format(val_loss/exp))
    
#         grad_weights3 = 0
#         grad_weights2 = 0
#         grad_weights1 = 0
#         grad_bias3 = 0
#         grad_bias2 = 0
#         grad_bias1 = 0

#### Architecture: Shape it as you want ####

# Architecture
# conv batchnorm relu (stride 2, 10 feature maps) - > 16
# conv batchnorm relu (stride 2, 10 feature maps) - > 8
# conv batchnorm relu (stride 2, 10 feature maps) - > 4
# conv batchnorm relu (stride 2, 10 feature maps) - > 2
# conv batchnorm relu (stride 2, 10 feature maps) - > 1
# softmax
# y.squeeze()

# We essentially only want the gradient of the loss with respect to the weights, so we can perform grad descent.
# We don't need anything else!


RuntimeError: shape '[1, 1, 1]' is invalid for input of size 9

In [301]:
import torch
x = torch.tensor([1.5], requires_grad=True)
a = torch.nn.Parameter(torch.tensor([2.]))
b = torch.nn.Parameter(torch.tensor([10.]))
y = x*a
z = y+0.5*b
temp = z.backward()
print('gradients of a: %0.2f and b: %0.2f' % (a.grad.item(), b.grad.item()))

gradients of a: 1.50 and b: 0.50


In [3]:
num_correct = 0
num_examples = 200
rand = 1000
import numpy as np
import tensorflow as tf
import torch
from torch.autograd import Variable
import cv2
import IPython
y = np.zeros((10,num_examples))
val_loss = 0
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
def imshow(img):
    _,ret = cv2.imencode('.jpg', img)
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)
x_input = x_test[rand:num_examples+rand,:,:]
x_input = torch.FloatTensor(x_input)
for i in range(y.shape[1]):
    y[y_test[i+rand],i] = 1
    y_val = Variable(torch.FloatTensor(y))
for index in range(num_examples):
    print(index)
    X = x_input[index,:,:]
    y = np.array(torch.Tensor(y_val[:,index]))
    X = torch.reshape(X,(1,X.shape[0],X.shape[1]))
    # X = torch.reshape(y,(1,X.shape[0],X.shape[1]))
    X_im2col,imX1 = im2col2d(X,kernel_shape1,stride1,im_needed = False,shape_specified = True)
    X_1 = relu(BatchNorm(torch.t(X_im2col).mm(weights1) + bias1))
    X_1 = torch.reshape(X_1,output_shape1)
    X_1_im2col, imX2 = im2col2d(X_1,kernel_shape2,stride2,im_needed = False,shape_specified = True)
    X_2 = relu(BatchNorm(torch.t(X_1_im2col).mm(weights2) + bias2))
    X_2 = torch.reshape(X_2,output_shape2)
    X_2_im2col, imX3 = im2col2d(X_2,kernel_shape3,stride3,im_needed = False,shape_specified = True)
    X_3 = BatchNorm(torch.t(X_2_im2col).mm(weights3) + bias3)
    X_x = X_3.squeeze()
    y_pred = np.array(softmax(X_x))
    val_loss += cross_entropy(y_pred.squeeze(),y.squeeze())
    prediction = np.argmax(y_pred,axis=0)
    actual = np.argmax(y,axis=0)
    print("Prediction is: {}".format(prediction))
    if prediction == actual:
        num_correct += 1
    if prediction == 5:
        imshow(np.array(X.squeeze()))
accuracy = num_correct / num_examples
accuracy *= 100
print(accuracy)
print(val_loss/num_examples)

0
Prediction is: 9
1
Prediction is: 0
2
Prediction is: 8
3
Prediction is: 3
4
Prediction is: 1
5
Prediction is: 9
6
Prediction is: 7
7
Prediction is: 9
8
Prediction is: 1
9
Prediction is: 0
10
Prediction is: 4
11
Prediction is: 1
12
Prediction is: 9
13
Prediction is: 9
14
Prediction is: 7
15
Prediction is: 9
16
Prediction is: 2
17
Prediction is: 4
18
Prediction is: 9
19
Prediction is: 1
20
Prediction is: 3
21
Prediction is: 7
22
Prediction is: 9
23
Prediction is: 4
24
Prediction is: 9
25
Prediction is: 1
26
Prediction is: 6
27
Prediction is: 1
28
Prediction is: 3
29
Prediction is: 8
30
Prediction is: 1
31
Prediction is: 2
32
Prediction is: 5


33
Prediction is: 8
34
Prediction is: 0
35
Prediction is: 4
36
Prediction is: 2
37
Prediction is: 1
38
Prediction is: 1
39
Prediction is: 1
40
Prediction is: 1
41
Prediction is: 8
42
Prediction is: 3
43
Prediction is: 4
44
Prediction is: 9
45
Prediction is: 9
46
Prediction is: 3
47
Prediction is: 0
48
Prediction is: 4
49
Prediction is: 2
50
Prediction is: 4
51
Prediction is: 4
52
Prediction is: 9
53
Prediction is: 2
54
Prediction is: 1
55
Prediction is: 7
56
Prediction is: 2
57
Prediction is: 4
58
Prediction is: 9
59
Prediction is: 4
60
Prediction is: 4
61
Prediction is: 0
62
Prediction is: 9
63
Prediction is: 9
64
Prediction is: 2
65
Prediction is: 2
66
Prediction is: 3
67
Prediction is: 3
68
Prediction is: 9
69
Prediction is: 3
70
Prediction is: 3
71
Prediction is: 7
72
Prediction is: 3
73
Prediction is: 7
74
Prediction is: 8
75
Prediction is: 1
76
Prediction is: 2
77
Prediction is: 9
78
Prediction is: 6
79
Prediction is: 6
80
Prediction is: 4
81
Prediction is: 9
82
Prediction is: 3


154
Prediction is: 0
155
Prediction is: 2
156
Prediction is: 9
157
Prediction is: 9
158
Prediction is: 4
159
Prediction is: 9
160
Prediction is: 4
161
Prediction is: 6
162
Prediction is: 4
163
Prediction is: 4
164
Prediction is: 9
165
Prediction is: 9
166
Prediction is: 3
167
Prediction is: 4
168
Prediction is: 8
169
Prediction is: 9
170
Prediction is: 9
171
Prediction is: 7
172
Prediction is: 3
173
Prediction is: 9
174
Prediction is: 2
175
Prediction is: 7
176
Prediction is: 0
177
Prediction is: 2
178
Prediction is: 4
179
Prediction is: 1
180
Prediction is: 1
181
Prediction is: 1
182
Prediction is: 6
183
Prediction is: 9
184
Prediction is: 1
185
Prediction is: 9
186
Prediction is: 7
187
Prediction is: 8
188
Prediction is: 0
189
Prediction is: 1
190
Prediction is: 3
191
Prediction is: 7
192
Prediction is: 9
193
Prediction is: 1
194
Prediction is: 7
195
Prediction is: 0
196
Prediction is: 1
197
Prediction is: 6
198
Prediction is: 4
199
Prediction is: 6
64.5
1.3998752357065678


In [ ]:
# y_pred = softmax(X_2)

# z = cross_entropy(y,y_pred) = sum(-y*log(y_pred))

# dz/X_2 = softmax(X_2) - y

# X_2 = X_im2col2'*weights2                       X_2 is not softmaxed

# dX_2/dweights2 = X_im2col2'

# X_im2col2 = relu(bn(X_im2col1'*weights1)

# X_1 = col2im(X_im2col2)

# dz/weights2 = dz/dX_2*dX_2/dweights2

# dz/weights1 = dz/dX_2*dX_2/dX_im2col2*dX_im2col2/dweights1

# dX_2/X_im2col2 = weights2'

# dz/weights1 = dz/dX_2*weights2'*relu(X_im2col1')

# X_2'*X1_im2col' = dW

# dX_col = 

In [30]:
weights1, bias1 = weights['layer1']
weights2, bias2 = weights['layer2']
weights3, bias3 = weights['layer3']

In [16]:
print(accuracy)

72.39999999999999


In [15]:
import pickle
with open('Vanilla_CNN_MNIST_weights_80%val','wb') as f:
    pickle.dump(weights_bias,f)

In [9]:
a = X_1_im2col
a = a.detach().numpy()
a = np.array(a)
print(a.size)

2500


In [7]:
print(y_pred,y)

NameError: name 'y_pred' is not defined

In [14]:
weights_bias = {'layer1': [weights1,bias1],'layer2':[weights2,bias2],'layer3':[weights3,bias3]}

In [1]:
print(im2)

NameError: name 'im2' is not defined

In [269]:
import sys
np.set_printoptions(threshold=sys.maxsize)
torch.set_printoptions(threshold=5000)

In [5]:
torch.set_printoptions(threshold=5000)
